In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
from google.oauth2 import service_account
from dotenv import load_dotenv
import pandas as pd
import os




load_dotenv()  # reads .env file

KEY_NAME_FILE = os.getenv("KEY_NAME_FILE")
print(KEY_NAME_FILE)

# Path to your service account JSON key
KEY_PATH = os.path.join(os.getcwd(), KEY_NAME_FILE)

# GA4 Property ID
PROPERTY_ID = os.getenv("PROPERTY_ID")
print(PROPERTY_ID)


# Authenticate
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = BetaAnalyticsDataClient(credentials=credentials)


#Get pageviews and users by page path for the last 7 days.
request = RunReportRequest(
    property=f"properties/{PROPERTY_ID}",
    dimensions=[Dimension(name="pagePath")],
    metrics=[Metric(name="pageviews"), Metric(name="activeUsers")],
    date_ranges=[DateRange(start_date="7daysAgo", end_date="today")],
)

response = client.run_report(request)

# Print results
for row in response.rows:
    print({dim.name: val.string_value for dim, val in zip(request.dimensions, row.dimension_values)},
          {met.name: val.string_value for met, val in zip(request.metrics, row.metric_values)})


#Convert Results to Pandas DataFrame
data = []
for row in response.rows:
    row_data = {dim.name: val.string_value for dim, val in zip(request.dimensions, row.dimension_values)}
    row_data.update({met.name: val.string_value for met, val in zip(request.metrics, row.metric_values)})
    data.append(row_data)

df = pd.DataFrame(data)
print(df.head())


ag-sa-seahorse-2-key.json
419931629


E0000 00:00:1760443927.638395  145904 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


InvalidArgument: 400 Did you mean keyEvents? Field pageviews is not a valid metric.  For a list of valid dimensions and metrics, see https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema 